# Ficha de Expressões Regulares 2

### Conceitos mais avançados de expressões regulares

- `.` - corresponde a uma ocorrência de qualquer caracter (exceto '\n', geralmente).
- `\w` - corresponde a um caracter alfanumérico (a-z, A-Z, 0-9 ou _).
- `\W` - corresponde a um caracter **não** alfanumérico.
- `\s` - corresponde a um caracter de *whitespace* (' ', '\t', ou '\n', por exemplo).
- `\S` - corresponde a um caracter que não seja *whitespace*.
- `\d` - corresponde a um dígito.
- `\D` - corresponde a um caracter que não seja um dígito.
- `\btot\w+` - corresponde a uma palavra **começada** por "tot" (o token `\b` representa uma *word boundary*, ou seja, o limite entre um caracter alfanumérico e outro não alfanumérico). Por outras palavras, captura a palavra "totalidade" mas não a palavra "batota". O token `\b` também pode ser usado no fim de palavras.
- `a(?=b)` - corresponde a um caracter `a` que tenha à sua frente um caracter `b`, mas não captura o caracter `b`. (*positive lookahead*)
- `a(?!b)` - corresponde a um caracter `a` que **não** tenha à sua frente um caracter `b`, mas não captura o caracter seguinte. (*negative lookahead*)
- `a(?<=b)` - corresponde a um caracter `a` que tenha atrás de si um caracter `b`, mas não captura o caracter `b`. (*positive lookbehind*)
- `a(?<!b)` - corresponde a um caracter `a` que **não** tenha atrás de si um caracter `b`, mas não captura o caracter anterior. (*negative lookbehind*)


Podemos usar *grupos de captura* em expressões regulares para isolar segmentos da string capturada. Usamos parênteses para definir grupos de captura.

In [ ]:
import re
m = re.search(r'(2[0-3]|[0-1][0-9]):([0-5][0-9])', "13:49")

print(m.groups()) # conjunto dos grupos de captura
print(m.group(0)) # toda a string capturada
print(m.group(1)) # o primeiro grupo de captura

('13', '49')
13:49
13


O módulo re possui ainda *flags* que podemos usar nas suas funções. As mais úteis são:

- `re.I` ou `re.IGNORECASE`: faz uma correspondência *case insensitive*.
- `re.M` ou `re.MULTILINE`: os tokens de âncora `^` e `$` passam a corresponder ao início/fim de cada linha, em vez do início/fim de uma string.
- `re.S` ou `re.DOTALL`: o token `.` passa a corresponder também a um caracter `\n`.

Podemos usar estas flags da seguinte forma: `re.search(r'trans.*mar', "TRANSF\nORMAR", re.I | re.S)`

## Exercício 1 - Conversão de datas

Define a função `iso_8601` que converte as datas presentes numa string no formato DD/MM/AAAA para o formato ISO 8601 - AAAA-MM-DD, usando expressões regulares e grupos de captura.

In [268]:
texto = """A 03/01/2022, Pedro viajou para a praia com a sua família.
Eles ficaram hospedados num hotel e aproveitaram o sol e o mar durante toda a semana.
Mais tarde, no dia 12/01/2022, Pedro voltou para casa e começou a trabalhar num novo projeto.
Ele passou muitas horas no escritório, mas finalmente terminou o projeto a 15/01/2022."""

import re

def iso_8601(text: str):
  pattern = re.compile(r"(?P<day>\d{2})/(?P<month>\d{2})/(?P<year>\d{4})")
  return pattern.sub(lambda m: f"{m.group('year')}-{m.group('month')}-{m.group('day')}", text)

print(iso_8601(texto))

A 2022-01-03, Pedro viajou para a praia com a sua família.
Eles ficaram hospedados num hotel e aproveitaram o sol e o mar durante toda a semana.
Mais tarde, no dia 2022-01-12, Pedro voltou para casa e começou a trabalhar num novo projeto.
Ele passou muitas horas no escritório, mas finalmente terminou o projeto a 2022-01-15.


## Exercício 2 - Validação de ficheiros

Escreve um programa que lê uma lista de nomes de ficheiros e determina se cada nome é válido ou não. O nome de um ficheiro deve conter apenas caracteres alfanuméricos, hífens, underscores ou pontos, seguido de uma extensão (e.g., ".txt", ".png", etc.).

In [116]:
file_names = [
  "document.txt", # válido
  "file name.docx", # inválido
  "image_001.jpg", # válido
  "script.sh.txt", # válido 
  "test_file.txt", # válido
  "file_name.", # inválido
  "my_resume.docx", # válido
  ".hidden-file.txt", # válido
  "important-file.text file", # inválido --
  "file%name.jpg" # inválido 
]

def valid_filename(filename: str):
  pattern = re.compile(r"[\.\w+\-\_]+(\.\w+)")
  return pattern.fullmatch(filename) != None

for filename in file_names:
  print(valid_filename(filename))

True
False
True
True
True
False
True
True
False
False


### Alínea 2.1

Modifica o programa anterior para colocar os nomes de ficheiro válidos num dicionário, no qual as chaves deverão ser as extensões dos mesmos. Por outras palavras, agrupa os ficheiros por extensão.

In [138]:
def validate_filename_dict(filenames: list):
  ext = re.compile(r"[\.\w+\-\_]+(?P<extension>\.\w+)")
  res = dict()
  for filename in file_names:
    if valid_filename(filename) == True:
      if match := ext.search(filename):
        m_dict = match.groupdict()
        if m_dict["extension"] not in res.keys():
          res[m_dict["extension"]] = [filename]
        else:
          val = res.get(m_dict["extension"])
          val.append(filename)
          res[m_dict["extension"]] = val
  
  return res

print(validate_filename_dict(file_names))

{'.txt': ['document.txt', 'script.sh.txt', 'test_file.txt', '.hidden-file.txt'], '.jpg': ['image_001.jpg'], '.docx': ['my_resume.docx']}


## Exercício 3 - Conversão de nomes

Escreve um filtro de texto que converte cada nome completo de uma pessoa encontrada num texto fonte, no formato `PrimeiroNome SegundoNome [...] UltimoNome` para o formato `UltimoNome, PrimeiroNome`. Por exemplo, "Rui Vieira de Castro" passa a "Castro, Rui". Atenção aos conectores "de", "dos", etc.

In [278]:
texto = """Este texto foi feito por Sofia Guilherme Rodrigues dos Santos, com 
base no texto original de Pedro Rafael Paiva Moura, com a ajuda
dos professores Pedro Rangel Henriques e José João Antunes Guimarães Dias De Almeida.
Apesar de partilharem o mesmo apelido, a Sofia não é da mesma família do famoso
autor José Rodrigues dos Santos."""

def sub_name(text: str):
  pattern = re.compile(r" [A-Z]\w+ [\w \w]* [A-Z]\w+")
  return pattern.sub(lambda m: f"{' '+m.group(0).split(' ')[-1] + ', '  + m.group(0).split(' ')[1]}",text)
    
print(sub_name(texto))

Este texto foi feito por Santos, Sofia, com 
base no texto original de Moura, Pedro, com a ajuda
dos professores Almeida, Pedro.
Apesar de partilharem o mesmo apelido, a Sofia não é da mesma família do famoso
autor Santos, José.


## Exercício 4 - Códigos postais 2

Define uma função `codigos_postais` que recebe uma lista de códigos postais e divide-os com base no hífen. Ao contrário do exercício da ficha anterior, esta função pode receber códigos postais inválidos. A função deve devolver uma lista de pares e apenas processar cada linha uma vez.

In [295]:
lista = [
    "4700-000", # válido
    "9876543", # inválido
    "1234-567", # válido
    "8x41-5a3", # inválido
    "84234-12", # inválido
    "4583--321", # inválido
    "9481-025" # válido
]

def codigos_postais(cod_postais: list):
    pattern = re.compile(r"(?P<left>\d{4})-(?P<right>\d{3})")
    res = []
    for cod_postal in cod_postais:
        if match := pattern.match(cod_postal):
            res.append((match.group("left"),match.group("right")))
    return res

print(codigos_postais(lista))


[('4700', '000'), ('1234', '567'), ('9481', '025')]


## Exercício 5 - Expansão de abreviaturas

Escreve um filtro de texto que expanda as abreviaturas que encontrar no texto fonte no formato "/abrev".

In [296]:
abreviaturas = {
  "UM": "Universidade do Minho",
  "LEI": "Licenciatura em Engenharia Informática",
  "UC": "Unidade Curricular",
  "PL": "Processamento de Linguagens"
}

texto = "A /abrev{UC} de /abrev{PL} é muito fixe! É uma /abrev{UC} que acrescenta muito ao curso de /abrev{LEI} da /abrev{UM}."

def abrev(abrevs: dict, text: str):
    pattern = re.compile(r"/abrev{(?P<key>\w+)}")
    return pattern.sub(lambda m: f"{abrevs.get(m.group('key'))}",text)

print(abrev(abreviaturas,texto))

A Unidade Curricular de Processamento de Linguagens é muito fixe! É uma Unidade Curricular que acrescenta muito ao curso de Licenciatura em Engenharia Informática da Universidade do Minho.


## Exercício 6 - Matrículas

Define uma função `matricula_valida` que recebe uma string de texto e determina se esta contém uma matrícula válida. Uma matrícula segue o formato AA-BB-CC, no qual dois dos três conjuntos devem ser compostos por números e o terceiro por letras maiúsculas (por exemplo, 01-AB-23), ou o novo formato no qual dois dos conjuntos são compostos por letras maiúsculas e o terceiro por números (por exemplo, 89-WX-YZ). Os conjuntos podem ser separados por um hífen ou um espaço.

Extra: Garante que o mesmo separador é usado para separar os três conjuntos.

In [313]:
matriculas = [
    "AA-AA-AA", # inválida
    "LR-RB-32", # válida
    "1234LX", # inválida
    "PL 22 23", # válida
    "ZZ-99-ZZ", # válida
    "54-tb-34", # inválida
    "12 34 56", # inválida
    "42-HA BQ" # válida, mas inválida com o requisito extra
]

def matricula_valida(text:str):
    pattern1 = re.compile(r"^\d{2}[ -][A-Z]{2}[ -][A-Z]{2}$")
    pattern2 = re.compile(r"^[A-Z]{2}[ -]\d{2}[ -][A-Z]{2}$")
    pattern3 = re.compile(r"^[A-Z]{2}[ -][A-Z]{2}[ -]\d{2}$")
    pattern4 = re.compile(r"^[A-Z]{2}[ -]\d{2}[ -]\d{2}$")
    pattern5 = re.compile(r"^\d{2}[ -][A-Z]{2}[ -]\d{2}$")
    pattern6 = re.compile(r"^\d{2}[ -]\d{2}[ -][A-Z]{2}$")
    return (pattern1.fullmatch(text) != None or pattern2.fullmatch(text) != None or pattern3.fullmatch(text) != None or pattern4.fullmatch(text) != None or pattern5.fullmatch(text) != None or pattern6.fullmatch(text) != None) and text[2] == text[5]

    

for matricula in matriculas:
    print(matricula_valida(matricula))

False
True
False
True
True
False
False
False


## Exercício 7 - *Mad Libs*

O jogo *Mad Libs*, bastante comum em países como os Estados Unidos, consiste em pegar num texto com espaços para algumas palavras e preencher esses espaços de acordo com o tipo de palavra que é pedida.

Escreve um programa que lê um texto no formato *Mad Libs* e pede ao utilizador para fornecer palavras que completem corretamente o texto.

In [351]:
texto = """Num lindo dia de [ESTAÇÃO DO ANO], [NOME DE PESSOA] foi passear com o seu [EXPRESSÃO DE PARENTESCO MASCULINA]. 
Quando chegaram à [NOME DE LOCAL FEMININO], encontraram um [OBJETO MASCULINO] muito [ADJETIVO MASCULINO].
Ficaram muito confusos, pois não conseguiam identificar a função daquilo.
Seria para [VERBO INFINITIVO]? Tentaram perguntar a [NOME DE PESSOA FAMOSA], que também não sabia.
Desanimados, pegaram no objeto e deixaram-no no [NOME DE LOCAL MASCULINO] mais próximo. 
Talvez os [NOME PLURAL MASCULINO] de lá conseguissem encontrar alguma utilidade para aquilo."""

def madlibs(text: str):
  pattern = re.compile(r"\[(?P<text>.*?)\]")
  return pattern.sub(lambda m: f"{input(m.group(0))}", text)
  

print(madlibs(texto))

Num lindo dia de inverno, Carlos foi passear com o seu cão. 
Quando chegaram à China, encontraram um pilar muito grande.
Ficaram muito confusos, pois não conseguiam identificar a função daquilo.
Seria para voar? Tentaram perguntar a Tomas Shelby, que também não sabia.
Desanimados, pegaram no objeto e deixaram-no no passeio mais próximo. 
Talvez os passeios de lá conseguissem encontrar alguma utilidade para aquilo.


## Exercício 8 - Remoção de repetidos

Escreve um filtro de texto que sempre que encontrar no texto fonte uma palavra repetida elimina as repetições, ou seja, substitui a lista de palavras por 1 só palavra.

In [375]:
texto = "Qual Qual é a cor do cavalo cavalo branco de Napoleão Napoleão?"

def remove_dup(text: str):
    return re.sub(r"\b(\w+)\b\s+\1", r"\1", text)

print(remove_dup(texto))

Qual é a cor do cavalo branco de Napoleão?
